In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#loading train
df_train=pd.read_csv('./cleaned_train_dummy.csv')
#loading test
df_test=pd.read_csv('cleaned_test_dummy.csv')

In [3]:
##Setting index as ID columns
df_test= df_test.set_index(['Id'])
df_train = df_train.set_index(['Id'])

In [4]:
##Setting y and x to begin running models
Y_vals = pd.DataFrame(df_train['SalePrice']).copy()
del df_train['SalePrice']
del df_test['SalePrice']
X_train=df_train.copy()
X_test=df_test.copy()

In [8]:

from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [7]:
lasso_reg = linear_model.LassoCV(random_state=0, normalize=True, eps = 0.0001, max_iter=10000)
mod = lasso_reg.fit(X_train, np.ravel(Y_vals))
mod.score(X_train, np.ravel(Y_vals))
##score=0.93082405005697799

0.93082405005697799

In [12]:
elastic_reg = linear_model.ElasticNetCV(normalize=True,eps = 0.0001,max_iter=10000).fit(X_train, np.ravel(Y_vals))
elastic_reg.score(X_train, np.ravel(Y_vals))

0.92633655284288607